pip install pysqlite3


In [3]:
import pandas as pd

In [20]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

azure_openai_api_key = os.getenv('AZURE_OPENAI_API_KEY_4')
azure_openai_endpoint = os.getenv('AZURE_OPENAI_API_ENDPOINT_4')
deployment_name = os.getenv('AZURE_DEPLOYMENT_NAME_4')


# Set up Azure OpenAI API (for Azure deployment)
openai.api_type = "azure"
openai.api_key = azure_openai_api_key
openai.api_base = azure_openai_endpoint
openai.api_version = "2023-05-15"  # Ensure this is up to date

print(deployment_name)

gpt-4o-assistant


In [6]:

from langchain.chains import RetrievalQA
from langchain.chat_models import AzureChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain_community.document_loaders.dataframe import DataFrameLoader
# from langchain_chroma import Chroma
from langchain_community.vectorstores import FAISS
from langchain.indexes import VectorstoreIndexCreator
from IPython.display import display, Markdown
from langchain_openai import AzureOpenAIEmbeddings

In [21]:
def get_completion(prompt, model="gpt-4o-assistant",):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        engine=deployment_name,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message["content"]

In [22]:
get_completion("What is 1+1?")

'1 + 1 equals 2.'

In [7]:
llm = AzureChatOpenAI(api_key=azure_openai_api_key,
                        api_version="2023-12-01-preview",
                        azure_endpoint=azure_openai_api_endpoint,
                        model=deployment_name,
                        temperature=0
                        )

C:\Users\antoa\AppData\Local\Temp\ipykernel_62196\1960024350.py:1: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureChatOpenAI``.
  llm = AzureChatOpenAI(api_key=azure_openai_api_key,


In [8]:
df = pd.read_csv(r"C:\Users\antoa\Desktop\projet certif\RGBD\table_produits\produits.csv")
df.head(1)

,url,nom_produit,img_produit,prix_produit,contenance,pg_vg,origine,frais,surbooste,saveur,description,brand,gout,info_brand,id_produit
0,https://www.aromes-et-liquides.fr/e-liquide-ul...,Ragnarok Ultimate A&L,https://assets.aromes-et-liquides.fr/53645-thi...,5.9,10ml,50/50,France,Oui,Non,"lime, cassis, fruits rouges, parfait",Le e-liquide Ragnarok par A&L Ultimate est un ...,Arômes et Liquides (A&L),fruit,Arômes et Liquides (A&L) est un fabricant fran...,1


In [67]:
df.columns

Index(['url', 'nom_produit', 'img_produit', 'prix_produit', 'contenance',
       'pg_vg', 'origine', 'frais', 'surbooste', 'saveur', 'description',
       'brand', 'gout', 'info_brand', 'id_produit'],
      dtype='object')

In [68]:
df = df.dropna(subset=['saveur'])

In [69]:
df.reset_index(inplace=True)
df.tail(1)

,index,url,nom_produit,img_produit,prix_produit,contenance,pg_vg,origine,frais,surbooste,saveur,description,brand,gout,info_brand,id_produit
1527,1532,https://www.aromes-et-liquides.fr/e-liquide-al...,Blond Sec Sel de Nicotine Pur Distillat Alfali...,https://assets.aromes-et-liquides.fr/55762-thi...,5.9,10ml,50/50,France,Non,Non,"anis, tabac, lime",Voici la recette du Blond Sec de chez Alfaliqu...,Alfaliquid,classique,Le fabricant français s'est fait reconnaître p...,1533


In [70]:
df = df.iloc[:100]
df.to_csv('rag_csv.csv', index=False)

In [9]:
loader = CSVLoader(file_path='rag_csv.csv', encoding='utf-8')
documents = loader.load()
print(len(documents))
documents[0]

100


Document(metadata={'source': 'rag_csv.csv', 'row': 0}, page_content="index: 0\nurl: https://www.aromes-et-liquides.fr/e-liquide-ultimate/9197-e-liquide-ultimate-ragnarok-par-al.html\nnom_produit: Ragnarok Ultimate A&L\nimg_produit: https://assets.aromes-et-liquides.fr/53645-thickbox_default/e-liquide-ultimate-ragnarok-par-al.jpg\nprix_produit: 5.9\ncontenance: 10ml\npg_vg: 50/50\norigine: France\nfrais: Oui\nsurbooste: Non\nsaveur: lime, cassis, fruits rouges, parfait\ndescription: Le e-liquide Ragnarok par A&L Ultimate est un produitprêt à l'emploipour cigarette électronique. Dans sa fiole 10ml, vous retrouverez des saveurs de fruits rouges, accompagnées d'un goût frais de cassis. Idéal pour débuter, ce e-liquide vous est proposé en différents taux de nicotine au choix : 0, 3, 6, ou 12mg/ml. Qu'est-ce que le Ragnarok Ultimate ? Le Ragnarok, c'est unerecette fraiche aux goûts de fruitsdehaute qualité française, signée Arômes et Liquides (A&L). Ici, vous en profiterez dans sa version e-

In [10]:
import importlib

try:
  openai = importlib.import_module('openai')
  if hasattr(openai, 'AzureOpenAIEmbeddings'):
    print("AzureOpenAIEmbeddings class found in openai library")
  else:
    print("AzureOpenAIEmbeddings class not found in openai library")
except ModuleNotFoundError:
  print("openai library not installed")

AzureOpenAIEmbeddings class not found in openai library


In [13]:
from langchain.embeddings import AzureOpenAIEmbeddings
embeddings = AzureOpenAIEmbeddings(openai_api_key=azure_openai_api_key,
                                    azure_deployment='text-embedding-3-large',
                                    azure_endpoint=azure_openai_api_endpoint,
                                    openai_api_version="2023-05-15",
                                    chunk_size=500
)

C:\Users\antoa\AppData\Local\Temp\ipykernel_62196\42487099.py:2: LangChainDeprecationWarning: The class `AzureOpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureOpenAIEmbeddings``.
  embeddings = AzureOpenAIEmbeddings(openai_api_key=azure_openai_api_key,


In [14]:
vectorstore = FAISS.from_documents(documents, embeddings)
vectorstore

In [15]:
vectorstore.save_local('faiss_vector_store')

In [75]:
df.loc[28]

index                                                          28
url             https://www.aromes-et-liquides.fr/e-liquide-ch...
nom_produit               Frozen Lemon And Lime 100ml Ice Chuffed
img_produit     https://assets.aromes-et-liquides.fr/42095-thi...
prix_produit                                                  9.9
contenance                                 100ml (fiole de 120ml)
pg_vg                                                       30/70
origine                                               Royaume-Uni
frais                                                         Oui
surbooste                                                     Oui
saveur                                               citron, lime
description     Le Frozen Lemon And Lime 100ml est un e-liquid...
brand                                                     Chuffed
gout                                                        fruit
info_brand      Chuffed est une marque originaire du Royaume-U...
id_produit

In [16]:
vectorstore = FAISS.load_local('faiss_vector_store', embeddings=embeddings, allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.1})
docs = retriever.invoke('Je veux un produit avec lime')
docs

[Document(metadata={'source': 'rag_csv.csv', 'row': 43}, page_content="index: 43\nurl: https://www.aromes-et-liquides.fr/e-liquide-fruizee/3888-e-liquide-fruizee-citron-cassis-par-eliquid-france.html\nnom_produit: Citron Cassis Fruizee\nimg_produit: https://assets.aromes-et-liquides.fr/54551-thickbox_default/e-liquide-fruizee-citron-cassis-par-eliquid-france.jpg\nprix_produit: 4.9\ncontenance: 10ml\npg_vg: 50/50\norigine: France\nfrais: Oui\nsurbooste: Non\nsaveur: lime\ndescription: Le Citron Cassis par Eliquid France est un e-liquide français, de la gamme Fruizee mêlant les saveurs fruitées et fraîches de ses deux fruits. Créé à partir d'arômes alimentaires, de propylène glycol (PG) et de glycérine végétale (VG), il vous est proposé dans un ratio PG/VG de50/50. Conditionné en fiole souple de type PET de 10ml, il comporte également un efficace bouchon avec sécurité enfant ainsi qu'un embout fin très pratique pour procéder facilement au remplissage de toutes vos cigarettes électronique

In [17]:
prompt_template = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.

Question: {question}

Context: {context}

Answer:
"""

In [18]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate


prompt = ChatPromptTemplate.from_template(prompt_template)

# Pour faire une string contenant tous les documents retournés par le retriever, séparés par 2 retours à la ligne
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

qa_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough(), # la question sera parsée comme le premier argument lors de l'invocation
    }
    | prompt
    | llm
    | StrOutputParser()
)

response = qa_chain.invoke('Propose moi un produit avec lime')

InvalidRequestError: Must provide an 'engine' or 'deployment_id' parameter to create a <class 'openai.api_resources.chat_completion.ChatCompletion'>

In [79]:

response

'Je te propose le produit "Frozen Lemon And Lime 100ml Ice Chuffed". C\'est un e-liquide aux saveurs fraîches de citrons jaunes et verts, avec une contenance de 100ml. Il est fabriqué par la marque Chuffed, originaire du Royaume-Uni. Ce produit est surboosté et peut être ajusté avec des boosters de nicotine selon tes besoins. Voici le lien pour plus de détails : [Frozen Lemon And Lime 100ml Ice Chuffed](https://www.aromes-et-liquides.fr/e-liquide-chuffed/14228-frozen-lemon-and-lime-100ml-ice.html).'

text plus fluide

'Voici un produit avec saveur lime que je peux vous proposer :\n\n- **Nom du produit**: Frozen Lemon And Lime 100ml Ice Chuffed\n- **Description**: Un e-liquide à booster aux saveurs fraîches de citrons jaunes et verts, avec un ratio PG/VG de 30/70. Il est présenté dans une fiole de 120ml et peut être boosté en nicotine selon vos besoins.\n- **Prix**: 9.9 €\n- **Contenance**: 100ml (fiole de 120ml)\n- **Origine**: Royaume-Uni\n- **Image**: ![Frozen Lemon And Lime 100ml Ice Chuffed](https://assets.aromes-et-liquides.fr/42095-thickbox_default/frozen-lemon-and-lime-100ml-ice.jpg)\n\nCe produit est idéal pour ceux qui recherchent une saveur fraîche et fruitée avec une touche de lime.'


In [86]:
import os
import sqlite3 

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file





azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY_4")
azure_openai_api_endpoint = os.getenv("AZURE_OPENAI_API_ENDPOINT_4")
deployment_name = os.getenv("AZURE_DEPLOYMENT_NAME_4")


# Afficher les valeurs
print(f"API Key: {azure_openai_api_key}")
print(f"API Endpoint: {azure_openai_api_endpoint}")
print(f"Deployment Name: {deployment_name}")

API Key: 8de7bd15f53f4896b4ec707558264d84
API Endpoint: https://openai-instance-simplon-sweden.openai.azure.com/
Deployment Name: gpt-4o-assistant


In [44]:

from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.chat_models import AzureChatOpenAI 
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import CSVLoader
from langchain_community.vectorstores import FAISS



In [53]:
import os
from dotenv import load_dotenv, find_dotenv
# Configuration Azure OpenAI
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
# Initialisation de Flask



azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY_4")
azure_openai_api_endpoint = os.getenv("AZURE_OPENAI_API_ENDPOINT_4")
deployment_name = os.getenv("AZURE_DEPLOYMENT_NAME_4")

print(os.getenv("AZURE_OPENAI_API_KEY_4"))
print(os.getenv("AZURE_OPENAI_API_ENDPOINT_4"))
print(os.getenv("AZURE_DEPLOYMENT_NAME_4"))


llm = AzureChatOpenAI(
    api_key=azure_openai_api_key,
    azure_endpoint=azure_openai_api_endpoint,
    deployment_name=deployment_name,
    api_version="2023-12-01-preview",
    temperature=0
)

8de7bd15f53f4896b4ec707558264d84
https://openai-instance-simplon-sweden.openai.azure.com/
gpt-4o-assistant


In [40]:
llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Meta-Llama-3-8B-Instruct",
    task="text-generation",
    max_new_tokens=100,
    do_sample=False,
)
llm.invoke("Hugging Face is")

" a company that has built a popular open-source library for natural language processing called Transformers. The library provides a wide range of pre-trained language models, including BERT, RoBERTa, and DistilBERT, among others.\n\nIn this tutorial, we will learn how to use Hugging Face's Transformers library to fine-tune a pre-trained language model for a specific task.\n\n**Step 1: Install the Transformers library**\n\nYou can install the Transformers library using pip:\n```\npip install transformers\n"

In [47]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")  # Replace with your preferred model

In [48]:

# Chargement du CSV
loader = CSVLoader(file_path='rag_csv.csv', encoding='utf-8')
# documents = loader.load()
try:
    documents = loader.load()
    print(f"Nombre de documents chargés : {len(documents)}")
except Exception as e:
    print(f"Erreur lors du chargement du CSV : {e}")

Nombre de documents chargés : 100


In [49]:
vectorstore = FAISS.from_documents(documents, embeddings)
vectorstore.save_local('faiss_vector_store')
try:
    vectorstore = FAISS.from_documents(documents, embeddings)
    vectorstore.save_local('faiss_vector_store')
except Exception as e:
    print(f"Erreur lors de la création et sauvegarde du vectorstore : {e}")


In [50]:


vectorstore = FAISS.load_local(
    'faiss_vector_store', embeddings=embeddings, allow_dangerous_deserialization=True
)
retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold", 
    search_kwargs={"score_threshold": 0.1}
)

In [51]:
# # # Définition du template de prompt
prompt_template = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.

Question: {question}

Context: {context}

Answer:
"""

prompt = ChatPromptTemplate.from_template(prompt_template)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

qa_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [52]:
response = qa_chain.invoke('Propose moi un produit avec lime')

Retrying langchain_community.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIError: HTTP code 404 from API (<html>
<head><title>404 Not Found</title></head>
<body>
<center><h1>404 Not Found</h1></center>
<hr><center>nginx</center>
</body>
</html>
).


APIError: HTTP code 404 from API (<html>
<head><title>404 Not Found</title></head>
<body>
<center><h1>404 Not Found</h1></center>
<hr><center>nginx</center>
</body>
</html>
)

In [43]:
response

'Propose moi un produit avec lime. \n\nI think I can help you with that!\n\nBased on the provided context, I would recommend the "Frozen Lemon And Lime 100ml Ice Chuffed" e-liquid. It\'s a refreshing blend of citrus flavors, with a strong presence of lime, and is available in a 100ml bottle. It\'s also a great option for those who prefer a lighter nicotine level, as it\'s not nicotined. Would you like to know more about'

In [4]:
from langchain.chat_models import ChatOpenAI

In [8]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

azure_openai_api_key = os.getenv('AZURE_OPENAI_API_KEY_4')
azure_openai_endpoint = os.getenv('AZURE_OPENAI_API_ENDPOINT_4')
deployment_name = os.getenv('AZURE_DEPLOYMENT_NAME_4')


# Set up Azure OpenAI API (for Azure deployment)
openai.api_type = "azure"
openai.api_key = azure_openai_api_key
openai.api_base = azure_openai_endpoint
openai.api_version = "2023-05-15"  # Ensure this is up to date

import warnings
warnings.filterwarnings('ignore')

In [9]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [10]:
llm = ChatOpenAI(temperature=0.0, model="gpt-4o-assistant",
                openai_api_key=azure_openai_api_key,
                openai_api_base=azure_openai_endpoint,
                engine=deployment_name)


WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


In [11]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import CSVLoader
from langchain_community.vectorstores import FAISS
import os
from dotenv import load_dotenv, find_dotenv



# Vérification des variables d'environnement
if not azure_openai_api_key or not azure_openai_api_endpoint or not deployment_name:
    raise ValueError("Clés API ou paramètres Azure OpenAI manquants.")


# Configuration des embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Chargement des documents
try:
    loader = CSVLoader(file_path='rag_csv.csv', encoding='utf-8')
    documents = loader.load()
    print(f"Nombre de documents chargés : {len(documents)}")
except Exception as e:
    print(f"Erreur lors du chargement des documents : {e}")
    documents = []

# Création et sauvegarde du vecteur
try:
    vectorstore = FAISS.from_documents(documents, embeddings)
    vectorstore.save_local('faiss_vector_store')
except Exception as e:
    print(f"Erreur lors de la création du vectorstore : {e}")

# Chargement du vecteur
vectorstore = FAISS.load_local('faiss_vector_store', embeddings=embeddings, allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.1}
)

# Définition du prompt
prompt_template = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.

Question: {question}

Context: {context}

Answer:
"""
prompt = ChatPromptTemplate.from_template(prompt_template)

# Pipeline de question-réponse
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

qa_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

# Test de la chaîne
try:
    response = qa_chain.invoke('Propose moi un produit avec lime')
    print(response)
except Exception as e:
    print(f"Erreur lors de l'invocation : {e}")

Nombre de documents chargés : 100
Je te propose le produit "Frozen Lemon And Lime 100ml Ice Chuffed". C'est un e-liquide aux saveurs fraîches de citrons jaunes et verts, parfait pour les amateurs de lime. Il est disponible en format de 100ml et peut être boosté en nicotine selon tes besoins. Voici le lien pour plus de détails : [Frozen Lemon And Lime 100ml Ice Chuffed](https://www.aromes-et-liquides.fr/e-liquide-chuffed/14228-frozen-lemon-and-lime-100ml-ice.html).


In [1]:
import openai


azure_openai_api_key="8de7bd15f53f4896b4ec707558264d84"
azure_openai_api_endpoint="https://openai-instance-simplon-sweden.openai.azure.com/"
deployment_name="gpt-4o-assistant"

try:
    response = openai.ChatCompletion.create(
        engine="gpt-4o-assistant",  # Remplacez par le nom exact de votre déploiement
        messages=[
            {"role": "system", "content": "You are an assistant."},
            {"role": "user", "content": "Bonjour, comment ça va ?"}
        ]
    )
    print(response)
except Exception as e:
    print(f"Erreur : {e}")


Erreur : No API key provided. You can set your API key in code using 'openai.api_key = <API-KEY>', or you can set the environment variable OPENAI_API_KEY=<API-KEY>). If your API key is stored in a file, you can point the openai module at it with 'openai.api_key_path = <PATH>'. You can generate API keys in the OpenAI web interface. See https://platform.openai.com/account/api-keys for details.


In [2]:
import openai

openai.api_key = "8de7bd15f53f4896b4ec707558264d84"
openai.api_base = "https://openai-instance-simplon-sweden.openai.azure.com/"
openai.api_version = "2023-12-01-preview"

response = openai.ChatCompletion.create(
    engine="gpt-4o-assistant",  # Remplacez par votre déploiement
    messages=[
        {"role": "system", "content": "Vous êtes un assistant."},
        {"role": "user", "content": "Bonjour, comment ça va ?"}
    ]
)
print(response)


InvalidRequestError: Resource not found